In [1]:
%matplotlib notebook
from tqdm import tqdm
import time
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import fftconvolve
import IPython
import pyroomacoustics as pra
from scipy.io import savemat

In [2]:
#加载自定义建筑的房间STL模型
from mpl_toolkits import mplot3d

import pyroomacoustics as pra

try:
    from stl import mesh
except ImportError as err:
    print(
        "The numpy-stl package is required for this example. "
        "Install it with `pip install numpy-stl`"
    )
    raise err

def main(file_path="3D_model_file/zj425.stl"):
    material = pra.Material(energy_absorption=0.2, scattering=0.1)

    # with numpy-stl
    the_mesh = mesh.Mesh.from_file(file_path)
    ntriang, nvec, npts = the_mesh.vectors.shape
    size_reduc_factor = 500.0  # to get a realistic room size (not 3km)

    # create one wall per triangle
    walls = []
    for w in range(ntriang):
        walls.append(
            pra.wall_factory(
                the_mesh.vectors[w].T / size_reduc_factor,
                material.energy_absorption["coeffs"],
                material.scattering["coeffs"],
            )
        )

    room = pra.Room(
            walls,
            fs=16000,
            max_order=3, #调整阶数
            ray_tracing=False,
            air_absorption=True,
        )
    return room
# 在这里手动设置文件路径
room = main(file_path="D:/MATLAB/Matlab_HaHa/Degree_project/3D_model_file/zj425.stl")
#加载自定义建筑的房间STL模型完成

In [3]:
len(room.walls)

24

In [4]:
room.is_inside([100, -20, -3])

False

In [5]:
#显示房间模型
fig, ax = room.plot();
ax.set_xlim([-20, 20])
ax.set_ylim([-30, 0])
ax.set_zlim([-12, 4])
fig.show()

<IPython.core.display.Javascript object>

In [6]:
def SDML7(d=0.1, r=[0, 0, 0]):
    mic_7 = np.array([[d / 2, -d / 2, 0, 0, 0, 0, 0],
                      [0, 0, d / 2, -d / 2, 0, 0, 0],
                      [0, 0, 0, 0, d / 2, -d / 2, 0]])

    # 将 r 转换为数组
    r = np.array(r)

    # 将 r 添加到 mic_7 中的每个元素
    mic_7 = mic_7 + r[:, np.newaxis]

    return mic_7

r=[0, -20, -3]
mic = SDML7(0.1,r)#第一个SRIR的位置是[0,-20,-3] 第二个SRIR的位置是[0, -8, -3] 中间值是[0, -14, -3]
print(mic)

[[  0.05  -0.05   0.     0.     0.     0.     0.  ]
 [-20.   -20.   -19.95 -20.05 -20.   -20.   -20.  ]
 [ -3.    -3.    -3.    -3.    -2.95  -3.05  -3.  ]]


In [7]:
for i in mic.T:
    print(i)

[  0.05 -20.    -3.  ]
[ -0.05 -20.    -3.  ]
[  0.   -19.95  -3.  ]
[  0.   -20.05  -3.  ]
[  0.   -20.    -2.95]
[  0.   -20.    -3.05]
[  0. -20.  -3.]


In [8]:
#添加声源和麦克风
# specify signal source
fs, signal = wavfile.read("audio_warehouse/singing.wav")    #arctic_a0010.wav")
room.add_source([0, -1.0, -7.0],signal=signal)
room.add_microphone_array(mic)
#R = pra.circular_2D_array(center=[0, -20, -3], M=6, phi0=0, radius=0.1)
#room.add_microphone_array(pra.MicrophoneArray(R, room.fs))
#room.add_microphone_array(np.c_[])

C:\Users\Perry Pan\AppData\Local\Temp\ipykernel_33620\205733844.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, signal = wavfile.read("audio_warehouse/singing.wav")    #arctic_a0010.wav")


In [9]:
#显示当前模型情况
fig, ax = room.plot()
ax.set_xlim([-20, 20])
ax.set_ylim([-30, 0])
ax.set_zlim([-12, 4])
fig.show()

<IPython.core.display.Javascript object>

In [10]:
img = room.sources[0].images

In [11]:
img

array([[ 0.],
       [-1.],
       [-7.]], dtype=float32)

In [12]:
# compute image sources
room.image_source_model()

# visualize 3D polyhedron room and image sources
fig, ax = room.plot(img_order=2)

<IPython.core.display.Javascript object>

In [13]:
room.plot_rir()
fig = plt.gcf()
#fig.set_size_inches(10, 20)

<IPython.core.display.Javascript object>

In [14]:
room.fs

16000

In [15]:
room.compute_rir()

In [16]:
#room.ray_tracing()

In [17]:
# show the room
room.plot(img_order=1)
plt.show()

<IPython.core.display.Javascript object>

In [18]:
t60 = pra.experimental.measure_rt60(room.rir[0][0], fs=room.fs, plot=True)
print(f"The RT60 is {t60 * 1000:.0f} ms")

ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
room.simulate()
print(room.mic_array.signals.shape)

In [ ]:
print(room.mic_array)

In [ ]:
# original signal
print("Original WAV:")
IPython.display.Audio(signal, rate=fs)

In [ ]:
print("Simulated propagation to first mic:")
IPython.display.Audio(room.mic_array.signals[0,:], rate=fs)

In [ ]:
print("Simulated propagation to first mic:")
IPython.display.Audio(room.mic_array.signals[0,:], rate=fs)

In [ ]:
room.rir

In [ ]:
# 导出 RIR 与 mic array 的坐标
#if np.array_equal(r, [0, -20, -3]):
#    rir_name = 'rir_20'
#    mic_position_name = 'mic_position_20'
#elif np.array_equal(r, [0, -8, -3]):
#    rir_name = 'rir_8'
#    mic_position_name = 'mic_position_8'
rir_name = 'rir'
mic_position_name = 'mic_position'
# 找到最小的数组长度
min_length = min(len(arr[0]) for arr in room.rir)

# 创建一个新的列表，其中包含所有数组的修剪版本
trimmed_rir = [arr[0][:min_length] for arr in room.rir]

# 现在所有的数组都具有相同的长度，可以保存到.mat文件中
savemat(rir_name+'.mat', {rir_name: trimmed_rir})
savemat(mic_position_name+'.mat', {mic_position_name: mic})


In [ ]:
for arr in room.rir:
    print(arr)
    print(len(arr[0]))
    print('!!!!!!!!!!!!!!!!')

In [ ]:
import numpy as np

import pyroomacoustics as pra

corner = np.array([[0, 0], [7, 0], [7, 5], [0, 5]]).T
room = pra.Room.from_corners(corner)
'''
corner: 房间的四个角。`pra.Room.from_corners()`的函数文档中表示四个角的声明顺序必须逆时针。
'''
fig, ax = room.plot()
ax.set_xlim([-1, 10])
ax.set_ylim([-1, 10])
fig.show()

In [ ]:
import numpy as np
import pyroomacoustics as pra
corner = np.array([[0, 0], [7, 0], [7, 5], [0, 5]]).T
room = pra.Room.from_corners(corner)
room.extrude(3.)
fig, ax = room.plot()
ax.set_xlim([-1, 10])
ax.set_ylim([-1, 10])
ax.set_zlim([-1, 4])
fig.show()
type(room)

In [ ]:
import pyroomacoustics as pra

# The desired reverberation time and dimensions of the room
rt60 = 0.5  # seconds
room_dim = [9, 7.5, 3.5]  # meters

# We invert Sabine's formula to obtain the parameters for the ISM simulator
e_absorption, max_order = pra.inverse_sabine(rt60, room_dim)

# Create the room
room = pra.ShoeBox(
    room_dim, fs=16000, materials=pra.Material(e_absorption), max_order=max_order
)
room

In [ ]:
room

In [ ]:
default_stl_path

In [ ]:
Path(__file__).parent

In [ ]:
import R_SRIR_G as RSG
h = RSG.Room_SRIR_Generater([0, -5,-5], [0 ,-8, -3],'1', mic_type='array', Order = 3);

In [ ]:
h